In [26]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [27]:
from proj1_helpers import *
from implementations import *

DATA_TRAIN_PATH = '../data/train.csv.zip'
DATA_TEST_PATH = '../data/test.csv.zip'

# Load data
data_train = pd.read_csv(DATA_TRAIN_PATH)
data_test = pd.read_csv(DATA_TEST_PATH) # NOTE: This is test for SUBMISSION, not test for training.

# Make Y binary [-1,1]
data_train.loc[data_train['Prediction'] == 's','Prediction'] = 1
data_train.loc[data_train['Prediction'] == 'b','Prediction'] = 0
data_train['Prediction'] = pd.to_numeric(data_train['Prediction'])

# Extract y
y_train = data_train['Prediction'].to_frame()
y = y_train.to_numpy()

# Get ids
id_train = data_train['Id'].to_frame()

# Get x
x_train = data_train.copy()
x_train.drop(['Id','Prediction'], 1, inplace=True) 
x = x_train.to_numpy()


## Hackz

In [94]:
# Initial survey of data
'''
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

mod = smf.logit(formula='Prediction ~  DER_mass_MMC + DER_mass_transverse_met_lep + DER_mass_vis + \
                         DER_pt_h + DER_deltaeta_jet_jet + DER_mass_jet_jet + \
                         DER_pt_ratio_lep_tau + DER_met_phi_centrality + DER_lep_eta_centrality + \
                         PRI_tau_pt + PRI_tau_eta + PRI_tau_phi + PRI_lep_pt + PRI_lep_eta + PRI_lep_phi + \
                         PRI_met + PRI_met_phi + PRI_met_sumet + PRI_jet_num + PRI_jet_leading_pt + \
                         PRI_jet_leading_eta + PRI_jet_leading_phi + PRI_jet_subleading_pt + \
                         PRI_jet_subleading_eta + PRI_jet_subleading_phi + PRI_jet_all_pt', data=data_train)
res = mod.fit()
print(res.summary())
'''

"\nfrom scipy import stats\nimport statsmodels.api as sm\nimport statsmodels.formula.api as smf\n\nmod = smf.logit(formula='Prediction ~  DER_mass_MMC + DER_mass_transverse_met_lep + DER_mass_vis +                          DER_pt_h + DER_deltaeta_jet_jet + DER_mass_jet_jet +                          DER_pt_ratio_lep_tau + DER_met_phi_centrality + DER_lep_eta_centrality +                          PRI_tau_pt + PRI_tau_eta + PRI_tau_phi + PRI_lep_pt + PRI_lep_eta + PRI_lep_phi +                          PRI_met + PRI_met_phi + PRI_met_sumet + PRI_jet_num + PRI_jet_leading_pt +                          PRI_jet_leading_eta + PRI_jet_leading_phi + PRI_jet_subleading_pt +                          PRI_jet_subleading_eta + PRI_jet_subleading_phi + PRI_jet_all_pt', data=data_train)\nres = mod.fit()\nprint(res.summary())\n"

## Using above
Above can be used to test different models for logistic regression. Test different combinations etc, also get parameters (coefficients). We can begin by only using the predictors with strong coefficients, for example "DER_pt_ratio_lep_tau". /Martin

### Model 1: Linear model, logistic regression, standardized, 10% train

In [28]:
#tx = np.c_[np.ones((y.shape[0], 1)), x]
# Normalize data
x_mean = np.mean(x)
x_std = np.std(x)
#print(x_mean)
x_norm = (x - x_mean)/x_std

# Extend data with more features
tx = build_poly_1D(x_norm, 1)

# split data
x_train,y_train,x_test,y_test = split_data(tx, y, 0.1, seed=1)

# initialize a model (Not tested yet)
initial_w = np.zeros((tx.shape[1], 1))

max_iters = 100000
gamma = 0.000001

# Use cross validation?
# Use ridge regression?

# Logistical Regression
# NOTE: Logistic loss is now normalized with size of valuation data
w, loss = logistic_regression(y_train.reshape(y_train.size,1), x_train, initial_w, max_iters, gamma)

# Test model
te_loss = calculate_logistic_loss(y_test, x_test, w)
y_pred = np.rint(sigmoid(x_test@w))
diff = y_pred - y_test.reshape(y_test.shape[0],1)
print("Model classified {} % correct".format((1 -  sum(abs(diff))/y_test.shape[0])[0]))


iter: 0/100000, loss = [[0.69314718]]



KeyboardInterrupt: 

### Model 1 results:
iter: 499999/500000, loss = [[0.52571624]]

Model classified 0.7393377777777778 % correct

### Model 2: Logistic regression SGD

In [58]:
x_mean = np.mean(x)
x_std = np.std(x)
#print(x_mean)
x_norm = (x - x_mean)/x_std

# Extend data with more features
tx = build_poly_1D(x_norm, 2)

# split data
x_train,y_train,x_test,y_test = split_data(tx, y, 0.1, seed=1)

# initialize a model (Not tested yet)
initial_w = np.zeros((tx.shape[1], 1))

max_iters = 100000
#gamma = 0.0002
gamma = 0.0000001

# Use cross validation?
# Use ridge regression?

# Logistical Regression
# NOTE: Logistic loss is now normalized with size of valuation data
w, loss = logistic_regression_SGD(y_train.reshape(y_train.size,1), x_train, initial_w, max_iters, gamma, 0.01)

# Test model
te_loss = calculate_logistic_loss(y_test, x_test, w)
y_pred = np.rint(sigmoid(x_test@w))
diff = y_pred - y_test.reshape(y_test.shape[0],1)
#print()
sys.stdout.write("Model classified {} % correct".format((1 -  sum(abs(diff))/y_test.shape[0])[0]))
sys.stdout.flush()

iter: 0/100000, loss = [[0.69314718]]

iter: 1000/100000, loss = [[0.6641653]]

iter: 2000/100000, loss = [[0.64421798]]

iter: 3000/100000, loss = [[0.63254531]]

iter: 4000/100000, loss = [[0.62890472]]

iter: 5000/100000, loss = [[0.62668246]]

iter: 6000/100000, loss = [[0.62661024]]

iter: 7000/100000, loss = [[0.62453799]]

iter: 8000/100000, loss = [[0.62383786]]

iter: 9000/100000, loss = [[0.62271655]]

iter: 10000/100000, loss = [[0.62220509]]

iter: 11000/100000, loss = [[0.62178964]]

iter: 12000/100000, loss = [[0.62119106]]

iter: 13000/100000, loss = [[0.61965313]]



KeyboardInterrupt: 

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)